In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [2]:
import torch
import albumentations

import numpy as np
import pandas as pd

import torch.nn as nn
from sklearn import metrics
from sklearn import model_selection
from torch.nn import functional as F
from tqdm import tqdm
# from wtfml.utils import EarlyStopping
# from wtfml.engine import Engine
# from wtfml.data_loaders.image import ClassificationLoader
from efficientnet_pytorch import EfficientNet
import pretrainedmodels

from albumentations.pytorch import ToTensor
from torchvision import transforms

### Porting stuff from mtfml for our use

In [3]:
import torch

import numpy as np

from PIL import Image
from PIL import ImageFile


ImageFile.LOAD_TRUNCATED_IMAGES = True


class ClassificationLoader:
    def __init__(self, image_paths, targets, resize, augmentations=None):
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        image = Image.open(self.image_paths[item])
        targets = self.targets[item]
        if self.resize is not None:
            image = image.resize(
                (self.resize[1], self.resize[0]), resample=Image.BILINEAR
            )
        image = np.array(image)
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.long),
        }


In [4]:
class AverageMeter:
    """
    Computes and stores the average and current value
    """

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [5]:
import torch
import numpy as np


class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.0001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):
        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(
                "EarlyStopping counter: {} out of {}".format(
                    self.counter, self.patience
                )
            )
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print(
                "Validation score improved ({} --> {}). Saving model!".format(
                    self.val_score, epoch_score
                )
            )
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [6]:
try:
    from apex import amp
    _apex_available = True
except ImportError:
    _apex_available = False

class Engine:
    
    @staticmethod
    def train(
        data_loader,
        model,
        optimizer,
        device,
        scheduler=None,
        accumulation_steps=1,
        use_tpu=False,
        fp16=False,
    ):
        if use_tpu and not _xla_available:
            raise Exception(
                "You want to use TPUs but you dont have pytorch_xla installed"
            )
        if fp16 and not _apex_available:
            raise Exception("You want to use fp16 but you dont have apex installed")
        if fp16 and use_tpu:
            raise Exception("Apex fp16 is not available when using TPUs")
        if fp16:
            accumulation_steps = 1
        losses = AverageMeter()
        predictions = []
        model.train()
        if accumulation_steps > 1:
            optimizer.zero_grad()
        tk0 = tqdm(data_loader, total=len(data_loader), disable=use_tpu)
        for b_idx, data in enumerate(tk0):
            for key, value in data.items():
                data[key] = value.to(device)
            if accumulation_steps == 1 and b_idx == 0:
                optimizer.zero_grad()
            _, loss = model(**data)

            if not use_tpu:
                with torch.set_grad_enabled(True):
                    if fp16:
                        with amp.scale_loss(loss, optimizer) as scaled_loss:
                            scaled_loss.backward()
                    else:
                        loss.backward()
                    if (b_idx + 1) % accumulation_steps == 0:
                        optimizer.step()
                        if scheduler is not None:
                            scheduler.step()
                        if b_idx > 0:
                            optimizer.zero_grad()
            else:
                loss.backward()
                xm.optimizer_step(optimizer)
                if scheduler is not None:
                    scheduler.step()
                if b_idx > 0:
                    optimizer.zero_grad()

            losses.update(loss.item(), data_loader.batch_size)
            tk0.set_postfix(loss=losses.avg)
        return losses.avg

    @staticmethod
    def evaluate(data_loader, model, device, use_tpu=False):
        losses = AverageMeter()
        final_predictions = []
        model.eval()
        with torch.no_grad():
            tk0 = tqdm(data_loader, total=len(data_loader), disable=use_tpu)
            for b_idx, data in enumerate(tk0):
                for key, value in data.items():
                    data[key] = value.to(device)
                predictions, loss = model(**data)
                predictions = predictions.cpu()
                losses.update(loss.item(), data_loader.batch_size)
                final_predictions.append(predictions)
                tk0.set_postfix(loss=losses.avg)
        return final_predictions, losses.avg

    @staticmethod
    def predict(data_loader, model, device, use_tpu=False):
        model.eval()
        final_predictions = []
        with torch.no_grad():
            tk0 = tqdm(data_loader, total=len(data_loader), disable=use_tpu)
            for b_idx, data in enumerate(tk0):
                for key, value in data.items():
                    data[key] = value.to(device)
                predictions, _ = model(**data)
                predictions = predictions.cpu()
                final_predictions.append(predictions)
        return final_predictions

In [7]:
class Net(nn.Module):
    def __init__(self, arch):
        super(Net, self).__init__()
        self.arch = arch
        in_features  = arch._fc.in_features
        self.arch._fc = nn.Linear(in_features=in_features, out_features=1, bias=True)

        
    def forward(self, image,targets):
        """
        No sigmoid in forward because we are going to use BCEWithLogitsLoss
        Which applies sigmoid for us when calculating a loss
        """
        batch_size, _, _, _ = image.shape
        x = image
        
        criterion = nn.BCEWithLogitsLoss()
        ### https://github.com/clovaai/CutMix-PyTorch/blob/master/train.py
        output = self.arch(x)
        loss = criterion(output, targets.view(-1,1).float())
        return output,loss

In [8]:
# arch = EfficientNet.from_pretrained('efficientnet-b3')
# arch
# arch._fc.in_features

In [9]:
def train(fold,bs,epochs,fp16,sz,arch='efficientnet-b3',debug=False):
    if sz is not None:
        sz = (sz,sz)
    else:
        sz = None
    
    _n = arch
    import os
    training_data_path = '../input/512x512-dataset-melanoma/512x512-dataset-melanoma/'
    df = pd.read_csv('../input/folds_08062020.csv')
    device = "cuda"
    epochs = epochs
    train_bs = bs
    valid_bs = bs//2

    df_train = df[df.fold != fold].reset_index(drop=True)
    df_valid = df[df.fold == fold].reset_index(drop=True)

    
    arch = EfficientNet.from_pretrained(arch)
    model = Net(arch=arch)  # New model for each fold
    model = model.to(device) 
    
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    train_aug = albumentations.Compose(
        [
            albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),
            albumentations.CoarseDropout(),
            albumentations.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=30),
            albumentations.Flip(p=0.5)
        ]
    )

    valid_aug = albumentations.Compose(
        [
            albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True)
        ]
    )

    if debug:
        train_images = df_train.image_id.values.tolist()[:250]
        train_images = [os.path.join(training_data_path, i + ".jpg") for i in train_images]
        train_targets = df_train.target.values[:250]

        valid_images = df_valid.image_id.values.tolist()[:250]
        valid_images = [os.path.join(training_data_path, i + ".jpg") for i in valid_images]
        valid_targets = df_valid.target.values[:250]
    else:
        train_images = df_train.image_id.values.tolist()
        train_images = [os.path.join(training_data_path, i + ".jpg") for i in train_images]
        train_targets = df_train.target.values

        valid_images = df_valid.image_id.values.tolist()
        valid_images = [os.path.join(training_data_path, i + ".jpg") for i in valid_images]
        valid_targets = df_valid.target.values
        
    train_dataset = ClassificationLoader(
        image_paths=train_images,
        targets=train_targets,
        resize=sz,
        augmentations=train_aug,
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=train_bs, shuffle=True, num_workers=4
    )

    valid_dataset = ClassificationLoader(
        image_paths=valid_images,
        targets=valid_targets,
        resize=sz,
        augmentations=valid_aug,
    )

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=valid_bs, shuffle=False, num_workers=4
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        patience=3,
        threshold=0.001,
        mode="max"
    )
    
    model, optimizer = amp.initialize(model, optimizer, opt_level='O1')
    es = EarlyStopping(patience=5, mode="max")
    
    for epoch in range(epochs):
        train_loss = Engine.train(train_loader, model, optimizer, device=device,fp16=fp16)
        predictions, valid_loss = Engine.evaluate(
            valid_loader, model, device=device
        )
        predictions = np.vstack((predictions)).ravel()
        auc = metrics.roc_auc_score(valid_targets, predictions)
        print(f"Epoch = {epoch}, AUC = {auc}")
        scheduler.step(auc)
        
        if sz is not None:
            ss = sz[0]
        else:
            ss = 512
            
        es(auc, model, model_path= "../models/model_arch_{}_sz_{}_fold_{}_epoch_{}_auc_{}.bin".format(_n,ss,fold,epoch,round(auc*100,2)))
        if es.early_stop:
            print("Early stopping")
            break

In [10]:
e = 20
debug= False
bs = 4
train(0,bs,e,True,1024,'efficientnet-b3',debug=debug)
train(1,bs,e,True,1024,'efficientnet-b3',debug=debug)
train(2,bs,e,True,1024,'efficientnet-b3',debug=debug)
train(3,bs,e,True,1024,'efficientnet-b3',debug=debug)
train(4,bs,e,True,1024,'efficientnet-b3',debug=debug)

Loaded pretrained weights for efficientnet-b3


  0%|          | 0/11662 [00:00<?, ?it/s]

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


  0%|          | 38/11662 [00:23<2:15:51,  1.43it/s, loss=0.447]

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


...

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0


 87%|████████▋ | 10165/11703 [1:33:56<13:24,  1.91it/s, loss=0.0929]

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


100%|██████████| 5823/5823 [05:02<00:00, 19.27it/s, loss=0.22]  


Epoch = 11, AUC = 0.9094980991141611
EarlyStopping counter: 5 out of 5
Early stopping


In [12]:
from IPython.display import FileLink, FileLinks
FileLinks('../models/')

../models/
  model_arch_efficientnet-b3_sz_512_fold_0_epoch_0_auc_92.72.bin
  model_arch_efficientnet-b3_sz_512_fold_0_epoch_1_auc_93.08.bin
  model_arch_efficientnet-b3_sz_512_fold_0_epoch_2_auc_93.25.bin
  model_arch_efficientnet-b3_sz_512_fold_1_epoch_0_auc_91.86.bin
  model_arch_efficientnet-b3_sz_512_fold_1_epoch_1_auc_92.79.bin
  model_arch_efficientnet-b3_sz_512_fold_1_epoch_2_auc_93.13.bin
  model_arch_efficientnet-b3_sz_512_fold_2_epoch_0_auc_92.49.bin
  model_arch_efficientnet-b3_sz_512_fold_2_epoch_1_auc_93.21.bin
  model_arch_efficientnet-b3_sz_512_fold_3_epoch_0_auc_90.89.bin
  model_arch_efficientnet-b3_sz_512_fold_3_epoch_1_auc_90.91.bin
  model_arch_efficientnet-b3_sz_512_fold_3_epoch_2_auc_92.63.bin
  model_arch_efficientnet-b3_sz_512_fold_4_epoch_0_auc_92.27.bin
  model_arch_efficientnet-b3_sz_512_fold_4_epoch_2_auc_92.36.bin
  model_arch_efficientnet-b3_sz_512_fold_4_epoch_5_auc_92.49.bin
  model_arch_efficientnet-b3_sz_1024_fold_0_epoch_0_auc_90.64.bin
  model_arch_efficientnet-b3_sz_1024_fold_0_epoch_2_auc_91.7.bin
  model_arch_efficientnet-b3_sz_1024_fold_0_epoch_3_auc_92.31.bin
  model_arch_efficientnet-b3_sz_1024_fold_1_epoch_0_auc_88.63.bin
  model_arch_efficientnet-b3_sz_1024_fold_1_epoch_1_auc_88.7.bin
  model_arch_efficientnet-b3_sz_1024_fold_1_epoch_2_auc_90.8.bin
  model_arch_efficientnet-b3_sz_1024_fold_1_epoch_4_auc_91.45.bin
  model_arch_efficientnet-b3_sz_1024_fold_1_epoch_6_auc_91.87.bin
  model_arch_efficientnet-b3_sz_1024_fold_1_epoch_7_auc_91.95.bin
  model_arch_efficientnet-b3_sz_1024_fold_1_epoch_8_auc_92.59.bin
  model_arch_efficientnet-b3_sz_1024_fold_1_epoch_12_auc_92.71.bin
  model_arch_efficientnet-b3_sz_1024_fold_2_epoch_0_auc_88.6.bin
  model_arch_efficientnet-b3_sz_1024_fold_2_epoch_1_auc_89.56.bin
  model_arch_efficientnet-b3_sz_1024_fold_2_epoch_2_auc_90.68.bin
  model_arch_efficientnet-b3_sz_1024_fold_2_epoch_3_auc_92.35.bin
  model_arch_efficientnet-b3_sz_1024_fold_2_epoch_5_auc_92.72.bin
  model_arch_efficientnet-b3_sz_1024_fold_2_epoch_10_auc_92.81.bin
  model_arch_efficientnet-b3_sz_1024_fold_3_epoch_0_auc_88.86.bin
  model_arch_efficientnet-b3_sz_1024_fold_3_epoch_1_auc_90.55.bin
  model_arch_efficientnet-b3_sz_1024_fold_3_epoch_2_auc_90.79.bin
  model_arch_efficientnet-b3_sz_1024_fold_3_epoch_4_auc_91.34.bin
  model_arch_efficientnet-b3_sz_1024_fold_3_epoch_5_auc_92.36.bin
  model_arch_efficientnet-b3_sz_1024_fold_4_epoch_0_auc_90.07.bin
  model_arch_efficientnet-b3_sz_1024_fold_4_epoch_2_auc_91.27.bin
  model_arch_efficientnet-b3_sz_1024_fold_4_epoch_4_auc_91.61.bin
  model_arch_efficientnet-b3_sz_1024_fold_4_epoch_6_auc_91.68.bin